# RNN Applications
- Time series predication
- Language modeling (Text generation)
- Text sentiment Analysis
- Named entity recongnition
- Translation
- Speech recognition
- Music Composition

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import sys

### Define different cells using nn module

In [2]:
# output is hidden_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
cell

RNN(4, 2, batch_first=True)

In [3]:
cell = nn.LSTM(input_size = 4, hidden_size = 2, batch_first = True)
cell

LSTM(4, 2, batch_first=True)

In [4]:
cell = nn.GRU(input_size = 4, hidden_size = 2, batch_first = True)
cell

GRU(4, 2, batch_first=True)

In [5]:
# inputs is (batch_size, seq_len, input_size) with batch_first = True 
# hidden is (num_layers, batch_size, hidden_size)

In [6]:
# feed letters as inputs
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [7]:
# batch_size, seq_len, input_size
inputs = torch.autograd.Variable(torch.Tensor([[h]]))
inputs.shape

torch.Size([1, 1, 4])

In [8]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 1, 2))
hidden.shape

torch.Size([1, 1, 2])

In [9]:
out , hidden = cell(inputs, hidden)
# Expact two values as output 
print(out) 
print(hidden)

tensor([[[0.1505, 0.0483]]], grad_fn=<TransposeBackward0>)
tensor([[[0.1505, 0.0483]]], grad_fn=<ViewBackward>)


### Build RNN for multiple inputs

In [10]:
# batch_size, seq_len, input_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
inputs = torch.autograd.Variable(torch.Tensor([[h, e, l, l, o]]))
inputs.shape

torch.Size([1, 5, 4])

In [11]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 1, 2))
out , hidden = cell(inputs, hidden)
out

tensor([[[ 0.9092,  0.6725],
         [ 0.6716, -0.3791],
         [ 0.8872, -0.6090],
         [ 0.9140, -0.6392],
         [ 0.2818, -0.2554]]], grad_fn=<TransposeBackward0>)

### Build Multiple Batch input

In [12]:
# batch_size, seq_len, input_size
cell = nn.RNN(input_size = 4, hidden_size = 2, batch_first = True)
inputs = torch.autograd.Variable(torch.Tensor([[h, e, l, l, o],
                                               [e, o, l, l, l],
                                               [l, l, e, e, l]]))
# Notice batch_first = True
# batch_size = 3
# seq_len = 5
# input_size = 4
inputs.shape

torch.Size([3, 5, 4])

In [13]:
# num_layers, batch_size, hidden_size (Initialize randome)
hidden = torch.autograd.Variable(torch.randn(1, 3, 2))
out , hidden = cell(inputs, hidden)
out

tensor([[[ 0.0713, -0.3400],
         [ 0.1548,  0.4288],
         [-0.1941,  0.1248],
         [ 0.1155,  0.1755],
         [-0.3772, -0.3337]],

        [[-0.8788,  0.0505],
         [ 0.1133, -0.4585],
         [ 0.2706,  0.4398],
         [-0.2489,  0.1431],
         [ 0.1305,  0.1578]],

        [[ 0.2721,  0.3261],
         [-0.1941,  0.1877],
         [ 0.0042,  0.1920],
         [-0.0868,  0.2282],
         [ 0.0148,  0.1562]]], grad_fn=<TransposeBackward0>)

### Train the network

We want to predict the following sequence:
- h --> i
- i --> h
- h --> e
- e --> l
- l --> l
- l --> o

There are five **input**  letters: h i e l  o 

There are fine **output** letters: h i e l  o 

Design our Loss function using **cross entropy**


In [15]:
# cell = nn.RNN(input_size = 5, hidden_size = 5, batch_first = True)
# labels = torch.autograd.Variable(torch.LongTensor(y_data))


In [16]:
# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

In [17]:
num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the RNN. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 1  # One by one
num_layers = 1  # one-layer rnn

In [18]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size, batch_first=True)

    def forward(self, hidden, x):
        # Reshape input (batch first)
        x = x.view(batch_size, sequence_length, input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)

    def init_hidden(self):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [19]:
model = Model()
print(model)
# Use cross entrpy loss and find the error
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr =0.1)

Model(
  (rnn): RNN(5, 5, batch_first=True)
)


In [26]:
# Lab 12 RNN
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility


idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=5, hidden_size=5, batch_first=True)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Reshape input
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)

        out, _ = self.rnn(x, h_0)
        return out.view(-1, num_classes)


# Instantiate RNN model
rnn = RNN(num_classes, input_size, hidden_size, num_layers)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = rnn(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

RNN(
  (rnn): RNN(5, 5, batch_first=True)
)
epoch: 1, loss: 1.693
Predicted string:  llllll
epoch: 2, loss: 1.523
Predicted string:  llllll
epoch: 3, loss: 1.393
Predicted string:  llllll
epoch: 4, loss: 1.263
Predicted string:  llllll
epoch: 5, loss: 1.146
Predicted string:  llllll
epoch: 6, loss: 1.055
Predicted string:  lhelll
epoch: 7, loss: 1.002
Predicted string:  ihelll
epoch: 8, loss: 0.965
Predicted string:  ihelll
epoch: 9, loss: 0.913
Predicted string:  ihelll
epoch: 10, loss: 0.879
Predicted string:  ihelll
epoch: 11, loss: 0.840
Predicted string:  ihelll
epoch: 12, loss: 0.805
Predicted string:  ihello
epoch: 13, loss: 0.779
Predicted string:  ihello
epoch: 14, loss: 0.758
Predicted string:  ihello
epoch: 15, loss: 0.738
Predicted string:  ihello
epoch: 16, loss: 0.717
Predicted string:  ihello
epoch: 17, loss: 0.694
Predicted string:  ihello
epoch: 18, loss: 0.667
Predicted string:  ihelll
epoch: 19, loss: 0.643
Predicted string:  ihelll
epoch: 20, loss: 0.647
Predicted s

C:\Users\Farshad\Miniconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:83: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
